In [2]:
#Load Environment Variables and API Key

import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#Retrieve Google API Key from Environment

google_api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
#Validate Google API Key

if google_api_key is None:
    print("Google API key not found. Please set the GOOGLE_API_KEY environment variable.")
else:
    print("Google API key loaded successfully.")

Google API key loaded successfully.


In [5]:
#Import required Libraries

from llama_index.core import (VectorStoreIndex, SimpleDirectoryReader, ServiceContext,
                              StorageContext, load_index_from_storage)
from llama_index.llms.google_genai import GoogleGenAI 
from llama_index.embeddings.google_genai import GoogleGenAIEmbedding
from google import genai
from IPython.display import Markdown, display

In [6]:
#Initialize Google Generative AI Client

client = genai.Client(api_key=google_api_key)

In [ ]:
#List all available Models

for model in client.models.list():
    display(Markdown(f" *Model*: {model.name} | {model.supported_actions}"))

In [ ]:
#List Models that support `generateContent`

for model in client.models.list():
    if 'generateContent' in model.supported_actions:
        display(Markdown(f" *Model*: {model.name} | {model.supported_actions}"))